In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [2]:
import xgboost

In [3]:
from xgboost import XGBClassifier

In [4]:
xgboost.__version__

'1.7.4'

In [5]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

In [6]:
features = features['pfam']

In [7]:
len(features)

183

In [8]:
le = LabelEncoder()

In [9]:
train_data = train_data[features]

In [10]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [11]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [12]:
#from https://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [13]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import make_scorer, f1_score
from scipy import stats
from xgboost import XGBClassifier
import numpy as np

# Assuming X and y are your features and labels

np.random.seed(7)

#{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0} 
clf = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)

# Define a custom scoring function for F1 score
f1_scorer = make_scorer(f1_score, average='weighted')

# Create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

# Perform cross-validation and store the F1 scores
f1_scores = cross_val_score(clf, X, y, cv=kf, scoring=f1_scorer)

# save F1 scores, mean, and standard error to csv file
data = {'F1_Scores': f1_scores}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('f1_scores_results_xgModel_noOutliers_newContaminationMetric_xg.csv', index=False)

In [14]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

# Assuming X and y are your features and labels

np.random.seed(7)

#{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0} #
clf = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)

# Define a custom scoring function for F1 score
def f1_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average=None)

# Create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

# Initialize an empty list to store F1 scores for each class
all_f1_scores = []

# Perform cross-validation and store the F1 scores for each fold
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    fold_f1_scores = f1_scorer(y_test, y_pred)
    all_f1_scores.append(fold_f1_scores)

# Convert the list of arrays to a numpy array
f1_scores = np.array(all_f1_scores)

# Save the F1 scores for each class separately
class_names = [f'Class_{i}' for i in range(f1_scores.shape[1])]
data = {class_name: f1_scores[:, i] for i, class_name in enumerate(class_names)}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('f1_scores_results_xgModel_noOutliers_newContaminationMetric_xg_byClass.csv', index=False)

In [15]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from scipy import stats

# Define a list of values of k to try
k_values = [6]

# Define a list of train sizes to iterate through
train_sizes = np.arange(0.05, 1, 0.05)  # Add or modify train sizes as needed

# Initialize dictionaries to store the results for each k and train size
results = {}

# Load your dataset and create X, y
# Assuming X and y are already defined

# Create an XGBoost classifier
#{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0} 
clf = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)

# Loop through different values of k
for k in k_values:
    print(f"\nNumber of splits (k): {k}")
    
    # Initialize dictionary to store results for this k
    k_results = {}

    # Loop through different train sizes
    for train_size in train_sizes:
        print(f"Train size: {train_size}")
        
        # Initialize StratifiedShuffleSplit cross-validator with the current train_size
        kf = StratifiedShuffleSplit(n_splits=k, train_size=train_size, random_state=7)

        # Perform cross-validation and store the results
        f1_scores = cross_val_score(clf, X, y, cv=kf, scoring='f1_weighted')

        # Store mean and standard error of the F1 scores for this train size
        k_results[train_size] = {'mean': np.mean(f1_scores), 'se': stats.sem(f1_scores)}

    # Store the results for this k in the overall results dictionary
    results[k] = k_results

# Convert the results to a DataFrame for easier analysis and visualization
results_df = pd.DataFrame.from_dict({(i, j): results[i][j] for i in results.keys() for j in results[i].keys()},
                                     orient='index')

# Save the results to a CSV file
results_df.to_csv('k_train_size_vs_f1_score_mean_se_newContaminationMetric_xg.csv')


Number of splits (k): 6
Train size: 0.05
Train size: 0.1
Train size: 0.15000000000000002
Train size: 0.2
Train size: 0.25
Train size: 0.3
Train size: 0.35000000000000003
Train size: 0.4
Train size: 0.45
Train size: 0.5
Train size: 0.55
Train size: 0.6000000000000001
Train size: 0.6500000000000001
Train size: 0.7000000000000001
Train size: 0.7500000000000001
Train size: 0.8
Train size: 0.8500000000000001
Train size: 0.9000000000000001
Train size: 0.9500000000000001


In [16]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from scipy import stats

# Define a list of values of k to try
k_values = [6]

# Define a list of train sizes to iterate through
train_sizes = np.arange(0.05, 1, 0.05)  # Add or modify train sizes as needed

# Initialize dictionaries to store the results for each k and train size
results = {}

# Load your dataset and create X, y
# Assuming X and y are already defined

#{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0} 
clf = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)

# Loop through different values of k
for k in k_values:
    print(f"\nNumber of splits (k): {k}")
    
    # Initialize dictionary to store results for this k
    k_results = {}

    # Loop through different train sizes
    for train_size in train_sizes:
        print(f"Train size: {train_size}")
        
        # Initialize dictionary to store results for this train size
        k_results[train_size] = {}

        # Initialize StratifiedShuffleSplit cross-validator with the current train_size
        kf = StratifiedShuffleSplit(n_splits=k, train_size=train_size, random_state=7)

        # Perform cross-validation and store the results
        for train_index, test_index in kf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Create and fit the XGBoost classifier
            clf.fit(X_train, y_train)

            # Predict on the test set
            y_pred = clf.predict(X_test)

            # Calculate F1 scores by class
            class_f1_scores = f1_score(y_test, y_pred, average=None)
            for i, score in enumerate(class_f1_scores):
                key = f'class_{i}_f1'
                if key not in k_results[train_size]:
                    k_results[train_size][key] = []
                k_results[train_size][key].append(score)

        # Calculate mean and standard error of the F1 scores for this train size
        for key in k_results[train_size]:
            if key.startswith('class_'):
                scores = np.array(k_results[train_size][key])
                k_results[train_size][key] = {'mean': np.mean(scores), 'se': stats.sem(scores)}

    # Store the results for this k in the overall results dictionary
    results[k] = k_results

# Convert the results to a DataFrame for easier analysis and visualization
results_df = pd.DataFrame.from_dict({(i, j): results[i][j] for i in results.keys() for j in results[i].keys()},
                                     orient='index')

# Save the results to a CSV file
results_df.to_csv('k_train_size_vs_f1_score_mean_se_newContaminationMetric_xg_by_class.csv')


Number of splits (k): 6
Train size: 0.05
Train size: 0.1
Train size: 0.15000000000000002
Train size: 0.2
Train size: 0.25
Train size: 0.3
Train size: 0.35000000000000003
Train size: 0.4
Train size: 0.45
Train size: 0.5
Train size: 0.55
Train size: 0.6000000000000001
Train size: 0.6500000000000001
Train size: 0.7000000000000001
Train size: 0.7500000000000001
Train size: 0.8
Train size: 0.8500000000000001
Train size: 0.9000000000000001
Train size: 0.9500000000000001
